# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
kgrid = [1, 1, 1]
Ecut = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is ``(ρ_\text{out} - ρ_\text{in})``, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns ``δρ``, which is added to ``ρ_\text{in}`` to yield ``ρ_\text{next}``,
the density for the next SCF step.

In [4]:
struct MyMixing
    α  # Damping parameter
end
MyMixing() = MyMixing(0.7)

function DFTK.mix(mixing::MyMixing, basis, δF::RealFourierArray, δF_spin=nothing; n_iter, kwargs...)
    if n_iter <= 2
        # Just do simple mixing on total density and spin density (if it exists)
        (mixing.α * δF, isnothing(δF_spin) ? nothing : mixing.α * δF_spin)
    else
        # Use the KerkerMixing from DFTK
        DFTK.mix(KerkerMixing(α=mixing.α), basis, δF, δF_spin; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -7.223106101006         NaN   3.32e-01    0.0 
  2   -7.245336729227   -2.22e-02   1.55e-01    0.0 
  3   -7.248646572784   -3.31e-03   6.96e-02    0.0 
  4   -7.249033211187   -3.87e-04   3.83e-02    0.0 
  5   -7.249157481846   -1.24e-04   2.13e-02    0.0 
  6   -7.249197315296   -3.98e-05   1.19e-02    0.0 
  7   -7.249210267211   -1.30e-05   6.77e-03    0.0 
  8   -7.249214581171   -4.31e-06   3.89e-03    0.0 
  9   -7.249216061104   -1.48e-06   2.27e-03    0.0 
 10   -7.249216585120   -5.24e-07   1.34e-03    0.0 
 11   -7.249216776512   -1.91e-07   8.02e-04    0.0 
 12   -7.249216848437   -7.19e-08   4.87e-04    0.0 
 13   -7.249216876146   -2.77e-08   2.99e-04    0.0 
 14   -7.249216887044   -1.09e-08   1.86e-04    0.0 
 15   -7.249216891404   -4.36e-09   1.16e-04    0.0 


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.